# AXON by examples

Since 0.8 [pyaxon](https://pypi.pythob.org/pypi/pyaxon) [python](http://www.python.org) library has function `display_html(values)`. It produces tabbed html view of representations of `values` (iterable of objects) in `AXON` in *compact*, *formatted with* and *without braces* forms. This allows to present pretty examples of representations in `AXON`.

Each example contains three parts:

1. loading of objects from text in `AXON`;
2. `pprint`'ing of the loaded objects;
3. presenting three forms of representations in `AXON` of the objects.

First import  module `axon`.

In [1]:
import axon
from pprint import pprint

### List

This example of lists of atomic values.

In [2]:
text = """
[1 3.14 1000D "abc абв" 2015-12-01 12:00-03:00 2015-12-01T12:00+03:00 ? ∞]
"""
vals = axon.loads(text)
pprint(vals)
axon.display_html(vals)

[[1,
  3.14,
  Decimal('1000'),
  'abc абв',
  datetime.date(2015, 12, 1),
  datetime.time(12, 0, tzinfo=datetime.timezone(datetime.timedelta(-1, 75600))),
  datetime.datetime(2015, 12, 1, 12, 0, tzinfo=datetime.timezone(datetime.timedelta(0, 10800))),
  nan,
  inf]]


:0: FutureWarning: IPython widgets are experimental and may change in the future.


### Dict

In [3]:
text = """
{name:"Alex" age:32}
"""
vals = axon.loads(text)
pprint(vals)
axon.display_html(vals)

[{'age': 32, 'name': 'Alex'}]


### Ordered dict

In [4]:
text = """
[name:"Alex" age:32]
"""
vals = axon.loads(text)
pprint(vals)
axon.display_html(vals)

[OrderedDict([('name', 'Alex'), ('age', 32)])]


### Tuple

In [5]:
text = """
(1 2005-06-15 11:00 4500d 0.75)
"""
vals = axon.loads(text)
pprint(vals)
axon.display_html(vals)

[(1, datetime.date(2005, 6, 15), datetime.time(11, 0), Decimal('4500'), 0.75)]


### Node

In [6]:
text = """
person {name:"Alex" age:34}
"""
vals = axon.loads(text)
pprint(vals)
axon.display_html(vals)

[person{name: 'Alex', age: 34}]


### Nodes mapped to objects

In [7]:
class Person:
    __slots__ = ('name', 'age')
    def __init__(self, name, age):
        self.name = name
        self.age = age
    def __repr__(self):
        return "Person(name=%r, age=%r)" % (self.name, self.age)

@axon.factory("person")
def person_factory(attrs, vals):
    return Person(**attrs)

@axon.reduce(Person)
def person_reduce(p):
    return axon.node("person", axon.odict([("name", p.name), ("age", p.age)]))

text = """
person {name:"Alex" age:34}
"""
vals = axon.loads(text, mode="strict")
pprint(vals)
axon.display_html(vals)

[Person(name='Alex', age=34)]


### Node with subnodes

In [8]:
text = """
tree {
    node {
        id: 1
        leaf {
            id: 2
            val: "abc"
        }
        leaf {
            id: 3
            val: "def"
        }
    }
    leaf {
        id: 4
        val: "ghi"
    }
}
"""
vals = axon.loads(text)
pprint(vals)
axon.display_html(vals)

[tree{node{id: 1 leaf{id: 2, val: 'abc'}, leaf{id: 3, val: 'def'}}, leaf{id: 4, val: 'ghi'}}]


### Node with references

Graph with list of attributed nodes and list of attributed edges.

In [9]:
text = """
graph {
    nodes: [
       &n1 node {id:1 val:4}
       &n2 node {id:2 val:7}
       &n3 node {id:3 val:2}
       &n4 node {id:4 val:5}
    ]
    edges: [
        edge {val:12 *n1 *n2}
        edge {val:8 *n1 *n4}
        edge {val:-2 *n2 *n3}
        edge {val:5 *n3 *n4}
    ]
}
"""
vals = axon.loads(text)
pprint(vals)
axon.display_html(vals)

[graph{nodes: [node{id: 1, val: 4}, node{id: 2, val: 7}, node{id: 3, val: 2}, node{id: 4, val: 5}], edges: [edge{val: 12 node{id: 1, val: 4}, node{id: 2, val: 7}}, edge{val: 8 node{id: 1, val: 4}, node{id: 4, val: 5}}, edge{val: -2 node{id: 2, val: 7}, node{id: 3, val: 2}}, edge{val: 5 node{id: 3, val: 2}, node{id: 4, val: 5}}]}]


### Stream of objects

In [10]:
text = """
[1 2015-12-01 12:00]
(1 2005-06-15 11:00 4500d 0.75)
{name:"Alex" age:32}
[name:"Alex" age:32]
person {name:"Alex" age:34}
"""
vals = axon.loads(text)
pprint(vals)
axon.display_html(vals)

[[1, datetime.date(2015, 12, 1), datetime.time(12, 0)],
 (1, datetime.date(2005, 6, 15), datetime.time(11, 0), Decimal('4500'), 0.75),
 {'age': 32, 'name': 'Alex'},
 OrderedDict([('name', 'Alex'), ('age', 32)]),
 person{name: 'Alex', age: 34}]


### Stream of key:val pairs 

In [11]:
text = """
list: [1 2015-12-01 12:00]
dict: (1 2005-06-15 11:00 4500d 0.75)
tuple: {name:"Alex" age:32}
odict: [name:"Alex" age:32]
node: person {name:"Alex" age:34}
"""
vals = axon.loads(text)
for key, val in vals:
    print(key, ':', val)
axon.display_html(vals)

list : [1, datetime.date(2015, 12, 1), datetime.time(12, 0)]
dict : (1, datetime.date(2005, 6, 15), datetime.time(11, 0), Decimal('4500'), 0.75)
tuple : {'name': 'Alex', 'age': 32}
odict : OrderedDict([('name', 'Alex'), ('age', 32)])
node : person{name: 'Alex', age: 34}
